In [9]:
import haralyzer as haralyzer
import csv as csv
import json as json
import os as os
import pandas as pd 
import multiprocessing
import traceback

def harToTimes(aTup):
    harPath = aTup[0]
    harFolder = aTup[1]

    firstParty = harPath.rsplit(".",1)[0]
    # print(firstParty)
    harJson = None
    with open(os.path.join(harFolder,harPath), mode="r") as aHar:
        try : 
            harJson = json.load(aHar)
        except json.JSONDecodeError as e:
            # print(e)
            traceback.print_exc()

    allEntries = harJson["log"]["entries"]
    allEntriesWithRequests = list(filter(lambda anEntry : "request" in anEntry,allEntries))
    thirdPartyEntries = allEntriesWithRequests
    times = []
    for entry in thirdPartyEntries: 
        times.append(entry["startedDateTime"])
    
    return times


harFolder = "../hars_games/"
harPathList : list[str] = [harPath for harPath in os.listdir(harFolder) if  os.path.isfile(os.path.join(harFolder,harPath)) ]    


pool = multiprocessing.Pool(14)
allTimes= []
harPathList = [(harPath, harFolder) for harPath in harPathList]
for timeList in pool.imap(harToTimes,harPathList):
    allTimes.extend(timeList)
pool.close()

resultsDf = pd.Series(allTimes)


In [10]:
resultsDf[resultsDf.duplicated()].shape

(18370,)